In [ ]:
#@title blank template
#@markdown This notebook from [github.com/matteoferla/pyrosetta_help](https://github.com/matteoferla/pyrosetta_help).

#@markdown It can be opened in Colabs via [https://colab.research.google.com/github/matteoferla/pyrosetta_help/blob/main/colabs/colabs-pyrosetta.ipynb](https://colab.research.google.com/github/matteoferla/pyrosetta_help/blob/main/colabs/colabs-pyrosetta.ipynb)

#@markdown It just for loading up PyRosetta

In [ ]:
#@title Installation
#@markdown Installing PyRosetta with optional backup to your drive (way quicker next time!).
#@markdown Note that PyRosetta occupies some 10 GB, so you'll need to be on the 100 GB plan of Google Drive (it's one pound a month).

#@markdown The following is not the real password. However, the format is similar.
username = 'boltzmann' #@param {type:"string"}
password = 'constant' #@param {type:"string"}
#@markdown Release to install:
_release = 'release-295' #@param {type:"string"}
#@markdown Use Google Drive for PyRosetta (way faster next time, but takes up space)
#@markdown (NB. You may be prompted to follow a link and possibly authenticate and then copy a code into a box
use_drive = True #@param {type:"boolean"}
#@markdown Installing `rdkit` and `rdkit_to_params` allows the creation of custom topologies (params) for new ligands
install_rdkit = True #@param {type:"boolean"}

# verify the username and password are correct without actually knowing them.
import hashlib
hashed_username = hashlib.sha256(username).hexdigest()
hashed_password = hashlib.sha256(password).hexdigest()
expected_hashed_username = 'cf6f296b8145262b22721e52e2edec13ce57af8c6fc990c8ae1a4aa3e50ae40e'
expected_hashed_password = '45066dd976d8bf0c05dc8dd4d58727945c3437e6eb361ba9870097968db7a0da'
assert hashed_username == expected_hashed_username, 'The hash of the username is not as expected'
assert hashed_password == expected_hashed_password, 'The hash of the password is not as expected'

import sys
import platform
import os
assert platform.dist()[0] == 'Ubuntu'
py_version = str(sys.version_info.major) + str(sys.version_info.minor)
if use_drive:
  from google.colab import drive
  drive.mount('/content/drive')
  _path = '/content/drive/MyDrive'
  os.chdir(_path)
else:
  _path = '/content'
if not any(['PyRosetta4.Release' in filename for filename in os.listdir()]):
  assert not os.system(f'curl -u {username}:{password} https://graylab.jhu.edu/download/PyRosetta4/archive/release/PyRosetta4.Release.python{py_version}.ubuntu/PyRosetta4.Release.python{py_version}.ubuntu.{_release}.tar.bz2 -o /content/a.tar.bz2')
  assert not os.system('tar -xf /content/a.tar.bz2')
assert not os.system(f'pip3 install -e {_path}/PyRosetta4.Release.python{py_version}.ubuntu.{_release}/setup/')
assert not os.system(f'pip3 install pyrosetta-help biopython')
if install_rdkit:
  assert not os.system(f'pip3 install rdkit-pypi rdkit-to-params')

import site
site.main()

In [ ]:
#@title Start PyRosetta
import pyrosetta
import pyrosetta_help as ph

no_optH = False #@param {type:"boolean"}
ignore_unrecognized_res=False  #@param {type:"boolean"}
load_PDB_components=False  #@param {type:"boolean"}
ignore_waters=False  #@param {type:"boolean"}


extra_options= ph.make_option_string(no_optH=no_optH,
                                  ex1=None,
                                  ex2=None,
                                  mute='all',
                                  ignore_unrecognized_res=ignore_unrecognized_res,
                                  load_PDB_components=load_PDB_components,
                                  ignore_waters=ignore_waters)


# capture to log
logger = ph.configure_logger()
pyrosetta.init(extra_options=extra_options)

In [ ]:
## Usual stuff
pose = ph.pose_from_alphafold2('P02144')

scorefxn = pyrosetta.get_fa_scorefxn()
relax = pyrosetta.rosetta.protocols.relax.FastRelax(scorefxn, 3)
movemap = pyrosetta.MoveMap()
movemap.set_bb(False)
movemap.set_chi(True)
relax.apply(pose)

In [ ]:
# Note that nglview does not work with Colabs but py3Dmol does.
# install py3Dmol
os.system(f'pip3 install py3Dmol')
import site
site.main()
# run
import py3Dmol
view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js',)
view.addModel(ph.get_pdbstr(pose),'pdb')
view.zoomTo()
view

In [ ]:
# Also note that RDKit Chem.Mol instances are not displays as representations by default.

In [ ]:
#@title Upload to Michelanglo (optional)
#@markdown [Michelanglo](https://michelanglo.sgc.ox.ac.uk/) is a website that
#@markdown allows the creation, annotation and sharing of a webpage with an interactive protein viewport.
#@markdown ([examples](https://michelanglo.sgc.ox.ac.uk/gallery)).
#@markdown The created pages are private —they have a 1 in a quintillion change to be guessed within 5 tries.

#@markdown Registered users (optional) can add interactive annotations to pages.
#@markdown A page created by a guest is editable by registered users with the URL to it
#@markdown (this can be altered in the page settings).
#@markdown Leave blank for guest (it will not add an interactive description):

username = ''  #@param {type:"string"}
password = ''  #@param {type:"string"}

import os
assert not os.system(f'pip3 install michelanglo-api')
import site
site.main()
from michelanglo_api import MikeAPI, Prolink
if not username:
  mike = MikeAPI.guest_login()
else:
  mike = MikeAPI(username, password)
    
page = mike.convert_pdb(pdbblock=ph.get_pdbstr(pose),
                        data_selection='ligand',
                        data_focus='residue',
                        )
if username:
     page.retrieve()
     page.description = '## Description\n\n'
     page.description += 'autogen bla bla'
     page.commit()
page.show_link()